In [182]:
import numpy as np
from __future__ import print_function
import random

In [216]:
sizes =[784, 30, 10]

Build weight matrices with Transposed shape for each x, y adjacent pairs in sizes

Use y,x instead of x, y as to get transposed weight matrix. 

In ij th element j-th neuron in l-th layer is connected to i-th neuron in (l + 1)-th layer.


In [217]:
weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1],sizes[1:])]

In [218]:
#biases are for every layer except input

biases = [np.random.randn(x, 1) for x in sizes[1:]] 

In [219]:
no_of_layers = len(sizes)

In [187]:
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0 + np.exp(-z))

# Derivative of activation function
def sigmoid_prime(z):
    """The sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [197]:
def feed_forward(activation):
    """Get output of neural net for given input"""
    global weights, biases
    for w,b in zip(weights, biases):
        activation = sigmoid(np.dot(w, activation) + b) 
        # we do w . a because w is transposed weight matrix, 
        # as each row in w which represent each weight in layer l+1 gets multiplied with
        # activations of neurons in layer l and added up
    return activation

In [207]:
def SGD(training_data, epochs, mini_batch_size, learning_rate, test_data=None):
    global weights, biases
    if test_data: test_len = len(test_data)
    train_len = len(training_data)
    for i in xrange(epochs):
        random.shuffle(training_data)
        mini_batches = [training_data[k:k+mini_batch_size] for k in range(0,train_len,mini_batch_size)]
        for mini_batch in mini_batches:
            update_mini_batch(mini_batch, learning_rate)
        if test_data:
            print("Epoch {0} : {1} accuracy in {2} test".format(
                i, evaluate(test_data), test_len))
        else:
            print("Epoch {0} complete".format(i))

In [199]:
def update_mini_batch(mini_batch, learning_rate):
    global weights, biases

    grad_batch_w = [np.zeros(w.shape) for w in weights]
    grad_batch_b = [np.zeros(b.shape) for b in biases]
    for x,y in mini_batch:
        delta_w, delta_b = back_prop(x, y)
        grad_batch_w = [gw + dgw for gw, dgw in zip(grad_batch_w, delta_w)]
        grad_batch_b = [gb + dgb for gb, dgb in zip(grad_batch_b, delta_b)]
    weights = [w - (learning_rate/len(mini_batch)) * gw for w, gw in zip(weights, grad_batch_w)]
    biases = [b - (learning_rate/len(mini_batch)) * gb for b, gb in zip(biases, grad_batch_b)]

In [200]:
def back_prop(x, y):
    global weights, biases

    grad_w = [np.zeros(w.shape) for w in weights]
    grad_b = [np.zeros(b.shape) for b in biases]
    activations = [x]
    activation = x
    weighted_sums = []
    for b,w in zip(biases, weights):
        z  = np.dot(w, activation) + b
        weighted_sums.append(z)
        activation = sigmoid(z)
        activations.append(activation)
    
    delta = (activations[-1] - y)* sigmoid_prime(weighted_sums[-1])
    grad_b[-1] = delta
    grad_w[-1] = np.dot(delta, activations[-2].transpose())
    
    for l in xrange(2, no_of_layers):
        z = weighted_sums[-l]
        z_prime = sigmoid_prime(z)
        delta = np.dot(weights[-l+1].transpose(), delta) * z_prime
        grad_b[-l] = delta
        grad_w[-l] = np.dot(delta, activations[-l-1].transpose())
    return (grad_w, grad_b)

In [201]:
def evaluate(test_data):
    test_results = [(np.argmax(feed_forward(x)), y) for (x,y) in test_data]
    return sum(int(x == y) for (x, y) in test_results)
    

In [202]:
import mnist_loader


In [215]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [220]:
SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0 : 9113 accuracy in 10000 test
Epoch 1 : 9269 accuracy in 10000 test
Epoch 2 : 9274 accuracy in 10000 test
Epoch 3 : 9386 accuracy in 10000 test
Epoch 4 : 9398 accuracy in 10000 test
Epoch 5 : 9413 accuracy in 10000 test
Epoch 6 : 9458 accuracy in 10000 test
Epoch 7 : 9440 accuracy in 10000 test
Epoch 8 : 9455 accuracy in 10000 test
Epoch 9 : 9460 accuracy in 10000 test
Epoch 10 : 9465 accuracy in 10000 test
Epoch 11 : 9453 accuracy in 10000 test
Epoch 12 : 9480 accuracy in 10000 test
Epoch 13 : 9485 accuracy in 10000 test
Epoch 14 : 9509 accuracy in 10000 test
Epoch 15 : 9481 accuracy in 10000 test
Epoch 16 : 9494 accuracy in 10000 test
Epoch 17 : 9493 accuracy in 10000 test
Epoch 18 : 9506 accuracy in 10000 test
Epoch 19 : 9494 accuracy in 10000 test
Epoch 20 : 9505 accuracy in 10000 test
Epoch 21 : 9485 accuracy in 10000 test
Epoch 22 : 9476 accuracy in 10000 test
Epoch 23 : 9502 accuracy in 10000 test
Epoch 24 : 9497 accuracy in 10000 test
Epoch 25 : 9500 accuracy in 10000 t

(50000, 784)

In [211]:
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [223]:
net = Network([784, 30, 10])

In [224]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9113 / 10000
Epoch 1: 9220 / 10000
Epoch 2: 9284 / 10000
Epoch 3: 9379 / 10000
Epoch 4: 9316 / 10000
Epoch 5: 9328 / 10000
Epoch 6: 9381 / 10000
Epoch 7: 9427 / 10000
Epoch 8: 9414 / 10000
Epoch 9: 9434 / 10000
Epoch 10: 9421 / 10000
Epoch 11: 9468 / 10000
Epoch 12: 9415 / 10000
Epoch 13: 9477 / 10000
Epoch 14: 9444 / 10000
Epoch 15: 9462 / 10000
Epoch 16: 9465 / 10000
Epoch 17: 9442 / 10000
Epoch 18: 9494 / 10000
Epoch 19: 9450 / 10000
Epoch 20: 9483 / 10000
Epoch 21: 9475 / 10000
Epoch 22: 9508 / 10000
Epoch 23: 9502 / 10000
Epoch 24: 9506 / 10000
Epoch 25: 9473 / 10000
Epoch 26: 9457 / 10000
Epoch 27: 9470 / 10000
Epoch 28: 9498 / 10000
Epoch 29: 9498 / 10000
